# Baseline Pipeline Walkthrough

This notebook runs the **baseline** edit loop: plan → edit → critic, with retries
until the edit is both realistic and minimal.


In [ ]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

ROOT = Path.cwd().resolve()
if (ROOT / "src").exists():
    repo_root = ROOT
elif (ROOT.parent / "src").exists():
    repo_root = ROOT.parent
else:
    repo_root = ROOT

sys.path.insert(0, str(repo_root))

from src.config import load_config
from src.integrations.openai_client import OpenAIPlanner
from src.integrations.replicate_client import ReplicateClient
from src.workflow.nodes.planning import plan_edit_node
from src.workflow.nodes.criticism import critique_generated_node
from src.workflow.state import AgentState
from src.utils.logging import configure_logging
from src.utils.paths import ensure_dir

In [ ]:
load_dotenv()
configure_logging(os.getenv("LOG_LEVEL", "INFO"))

In [ ]:
cfg = load_config()
project = cfg.project
workflow = cfg.workflow
agents = cfg.agents

raw_dir = Path(project.raw_dir)
baseline_dir = Path(project.baseline_dir)

ensure_dir(raw_dir)
images = sorted(raw_dir.glob("*.png")) + sorted(raw_dir.glob("*.jpg"))
if not images:
    raise FileNotFoundError(f"No images found in {raw_dir}")
image_path = images[0]

print("Workflow target:", workflow.target_attribute)
print("Baseline model:", workflow.baseline_model)
print("Selected image:", image_path)

In [ ]:
planner = OpenAIPlanner(
    model=workflow.openai_model,
    planner_prompt=agents.planner_prompt,
    critic_prompt=agents.critic_prompt,
)

replicate_client = ReplicateClient()

print("OpenAI model:", workflow.openai_model)
print("Baseline model:", workflow.baseline_model)

In [ ]:
state: AgentState = {
    "image_path": str(image_path),
    "target_attribute": workflow.target_attribute,
    "attempts": 0,
    "is_realistic": False,
    "is_minimal_edit": False,
}

for _ in range(workflow.max_attempts):
    state.update(
        plan_edit_node(
            state,
            planner=planner,
            target_attribute=workflow.target_attribute,
        )
    )
    edited_path = replicate_client.image_edit_baseline(
        model=workflow.baseline_model,
        image_path=state["image_path"],
        edit_plan=state["edit_plan"],
        target_object=state["target_object"],
        output_dir=baseline_dir,
        prompt_template=cfg.agents.baseline_edit_prompt,
    )
    state.update(
        {
            "edited_image_path": str(edited_path),
            "used_mock": replicate_client.last_baseline_used_mock,
        }
    )
    state.update(critique_generated_node(state, planner=planner))
    print(
        "attempt",
        state.get("attempts"),
        "realistic",
        state.get("is_realistic"),
        "minimal",
        state.get("is_minimal_edit"),
    )
    if state.get("is_realistic") and state.get("is_minimal_edit"):
        break

state